In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

## Dataset Load
overview1 = pd.read_csv('/content/drive/MyDrive/dataset/AirbnbOverview.csv')
overview2 = pd.read_csv('/content/drive/MyDrive/dataset/AirbnbOverview2.csv')
amenity = pd.read_csv('/content/drive/MyDrive/dataset/Amenities_20012025.csv')
calendar1 = pd.read_csv('/content/drive/MyDrive/dataset/Calendar_11122024.csv')
calendar2 = pd.read_csv('/content/drive/MyDrive/dataset/Calendar_18012025.csv')
description = pd.read_csv('/content/drive/MyDrive/dataset/Description_20012025.csv')
pricing1 = pd.read_csv('/content/drive/MyDrive/dataset/Pricing_11122024.csv')
pricing2 = pd.read_csv('/content/drive/MyDrive/dataset/Pricing_20012025.csv')
review = pd.read_csv('/content/drive/MyDrive/dataset/Reviews_20012025.csv')
data = pd.read_excel('/content/drive/MyDrive/dataset/DataDictionary.xlsx')

<ipython-input-2-219f1e68a5e7>:9: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  overview2 = pd.read_csv('/content/drive/MyDrive/dataset/AirbnbOverview2.csv')
<ipython-input-2-219f1e68a5e7>:13: DtypeWarning: Columns (2,3,4,5,10,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  description = pd.read_csv('/content/drive/MyDrive/dataset/Description_20012025.csv')
<ipython-input-2-219f1e68a5e7>:16: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  review = pd.read_csv('/content/drive/MyDrive/dataset/Reviews_20012025.csv')


In [ ]:
review.head()

,Country,Location,Airbnb_ListingID,ReviewID,Review_CreatedAt,ReviewLanguage,ReviewComments,ReviewComments_Localized,Rating,TypeOfTrip,...,Reviewer_Region,Reviewer_Country,Reviewer_MonthsOnAirbnb,Reviewer_YearsOnAirbnb,Host_ID,Host_firstName,Host_isSuperhost,Host_Response,Host_Response_Localized,RecordInserted
0,South Korea,Seoul,1132838252426923868,1.296856e+18,2024-11-24T07:41:07Z,ko,숙소 위치와 주차등은 좋아요<br/>내부가 좀 작고 <br/>창문에 달아두신 사진에...,The location and parking are good.<br/>It's a ...,4.0,NaN,...,NaN,NaN,NaN,3.0,452633347.0,선화,False,NaN,NaN,2025-01-20 17:06:38.369823
1,South Korea,Seoul,1132838252426923868,1.291773e+18,2024-11-17T07:22:56Z,ko,시진설명과 동일하게 좋았어요<br/>호스트분도 친절하고 빠른 응답에 만족합니다<br...,It was the same as the Sijin description.<br/>...,5.0,NaN,...,NaN,NaN,NaN,2.0,493828268.0,재현,False,NaN,NaN,2025-01-20 17:06:38.369849
2,South Korea,Seoul,1132838252426923868,1.247541e+18,2024-09-17T06:41:25Z,ko,사진과 동일한 장소에 편하게 지내다가 갑니다.,I had a comfortable stay in the same place as ...,5.0,NaN,...,NaN,NaN,NaN,6.0,227713029.0,경준,False,NaN,NaN,2025-01-20 17:06:38.369862
3,South Korea,Seoul,1132838252426923868,1.233039e+18,2024-08-28T06:29:07Z,ko,아늑하고 머물기 편했습니다~^^,It was cozy and easy to stay ~ ^ ^,5.0,NaN,...,NaN,NaN,NaN,5.0,278202446.0,찬규,False,NaN,NaN,2025-01-20 17:06:38.369870
4,South Korea,Seoul,1132838252426923868,1.217817e+18,2024-08-07T06:24:11Z,ko,깨끗하고 편의시설 잘 되어 있었습니다.,It was clean and well-equipped.,5.0,NaN,...,NaN,NaN,NaN,11.0,15211340.0,Siyoung,False,NaN,NaN,2025-01-20 17:06:38.369879


In [ ]:
## Dataset Concat
# 1은 2024년 12월 11일까지의 데이터 2는 2025년 1월 20일까지의 데이터 이 2개를 합치는 곳보다 데이터 2에 데이터 1을 합치는 방향이 나을듯
## overview
# 먼저 두 개의 데이터를 concat으로 합쳐서 전체 listing ID 빈도 확인
overview_all = pd.concat([overview1, overview2])
listing_counts = overview_all['Airbnb_ListingID'].value_counts()
# ID별로 몇 번 나오는지 판단해서 조건 분기
ids_appeared_twice = listing_counts[listing_counts == 2].index
ids_appeared_once = listing_counts[listing_counts == 1].index
# 조건에 맞게 데이터 선택
overview = pd.concat([
    overview2[overview2['Airbnb_ListingID'].isin(ids_appeared_twice)],
    overview1[overview1['Airbnb_ListingID'].isin(ids_appeared_once)]
], ignore_index=True)

## calendar
# 날짜 기준 설정
cutoff_date = pd.to_datetime("2025-01-18")
# datetime 타입으로 변환 (혹시 문자열일 경우 대비)
calendar1['Calendar_Date'] = pd.to_datetime(calendar1['Calendar_Date'])
calendar2['Calendar_Date'] = pd.to_datetime(calendar2['Calendar_Date'])
# 조건에 맞게 필터링
calendar1_filtered = calendar1[calendar1['Calendar_Date'] <= cutoff_date]
calendar2_filtered = calendar2[calendar2['Calendar_Date'] > cutoff_date]
# 병합
calendar = pd.concat([calendar1_filtered, calendar2_filtered], ignore_index=True)

## pricing
# 기준 날짜 설정
cutoff_date = pd.to_datetime("2025-01-20")

# datetime 변환 (문자열일 수 있으므로 안전하게 변환)
pricing1['Stay_Checkout'] = pd.to_datetime(pricing1['Stay_Checkout'])
pricing2['Stay_Checkout'] = pd.to_datetime(pricing2['Stay_Checkout'])

# 조건에 맞게 필터링
pricing1_filtered = pricing1[pricing1['Stay_Checkout'] < cutoff_date]
pricing2_filtered = pricing2[pricing2['Stay_Checkout'] >= cutoff_date]

# 병합
pricing = pd.concat([pricing1_filtered, pricing2_filtered], ignore_index=True)


In [ ]:
## overview & description

In [ ]:
overview = overview.dropna(subset=['Airbnb_ListingID'])
description = description.dropna(subset=['Airbnb_ListingID'])
overview = overview.dropna(subset=['Host_ID'])
description = description.dropna(subset=['Host_ID'])

In [ ]:
# Airbnb_ListingID와 Host_ID 모두 문자열로 통일
description['Airbnb_ListingID'] = pd.to_numeric(description['Airbnb_ListingID'], errors='coerce')
overview['Airbnb_ListingID'] = pd.to_numeric(overview['Airbnb_ListingID'], errors='coerce')

description['Host_ID'] = pd.to_numeric(description['Host_ID'], errors='coerce')
overview['Host_ID'] = pd.to_numeric(overview['Host_ID'], errors='coerce')
overview['Host_ID'] = overview['Host_ID'].astype(float)

In [ ]:
des_over = overview.merge(
    description,
    on=['Airbnb_ListingID'],
    how='inner',
    suffixes=('_review', '_desc')
)

In [ ]:
print(des_over.isnull().sum().to_string())

Country_review                          0
Location                                0
Airbnb_ListingID                        0
Lat                                     0
Lng                                     0
City                                 9317
LocalizedCity                           0
LocalizedNeighborhood                  16
LocalizedNeighbourhood_ML               0
Title                                   2
Bathrooms                              25
Bedrooms                               43
Beds                                   21
PersonCapacity                          0
Host_ID_review                          0
Host_FirstName                          2
Host_isSuperhost_review                 0
Is_NewListing                           2
isSuperhost                             0
PictureCount_review                     0
Currency                                0
BasicNightPrice                      2159
BasicNightPrice_Clean                2159
CleaningFee                       

In [ ]:
des_over['Lat']

,Lat
0,37.560309
1,37.536279
2,37.574548
3,37.556026
4,37.502280
...,...
19015,37.558390
19016,37.527900
19017,37.567250
19018,37.532950


In [ ]:
## calendar & pricing

In [ ]:
# 1. 날짜 타입 맞추기
calendar['Calendar_Date'] = pd.to_datetime(calendar['Calendar_Date'])
pricing['Stay_Checkin'] = pd.to_datetime(pricing['Stay_Checkin'])
pricing['Stay_Checkout'] = pd.to_datetime(pricing['Stay_Checkout'])

# 2. pricing 확장: 체크인~체크아웃 날짜 별로 풀기 (체크아웃 전날까지)
pricing_expanded = []
for _, row in pricing.iterrows():
    stay_dates = pd.date_range(
        start=row['Stay_Checkin'],
        end=row['Stay_Checkout'] - pd.Timedelta(days=1),
        freq='D'
    )
    for date in stay_dates:
        pricing_expanded.append({
            'Airbnb_ListingID': row['Airbnb_ListingID'],
            'Calendar_Date': date.normalize(),  # 날짜 단위만 사용
            'Guests': row['Guests'],  # 👈 Guests 인원 추가
            'Price_Per_Night': row['Price_Per_Night'],
            'Cleaning_Fee': row['Cleaning_Fee'],
            'Airbnb_Service_Fee': row['Airbnb_Service_Fee'],
            'Taxes': row['Taxes']
        })

pricing_expanded_df = pd.DataFrame(pricing_expanded)

# 3. calendar와 병합 (날짜 단위 정확히 매칭)
calendar['Calendar_Date'] = calendar['Calendar_Date'].dt.normalize()
cal_pri = calendar.merge(
    pricing_expanded_df,
    on=['Airbnb_ListingID', 'Calendar_Date'],
    how='left'
)

# 4. Price_Per_Night가 NaN인 행 제거
cal_pri = cal_pri[cal_pri['Price_Per_Night'].notna()].reset_index(drop=True)


In [ ]:
# Step 1: Airbnb_ListingID별 Cleaning_Fee 평균값 계산 (NaN 제외)
cleaning_fee_map = cal_pri.groupby('Airbnb_ListingID')['Cleaning_Fee'].transform(
    lambda x: x.fillna(method='ffill').fillna(method='bfill')
)

# Step 2: 해당 값을 우선 채우고, 여전히 NaN이면 0으로 대체
cal_pri['Cleaning_Fee'] = cal_pri['Cleaning_Fee'].fillna(cleaning_fee_map)
cal_pri['Cleaning_Fee'] = cal_pri['Cleaning_Fee'].fillna(0)


<ipython-input-14-b09cc6d19dc8>:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lambda x: x.fillna(method='ffill').fillna(method='bfill')


In [ ]:
pricing_expanded_df

,Airbnb_ListingID,Calendar_Date,Guests,Price_Per_Night,Cleaning_Fee,Airbnb_Service_Fee,Taxes
0,1000084126553197219,2025-01-06,2,80240,23600,NaN,NaN
1,1000084126553197219,2025-01-07,2,80240,23600,NaN,NaN
2,1000084126553197219,2025-01-08,2,80240,23600,NaN,NaN
3,1000084126553197219,2025-01-09,2,80240,23600,NaN,NaN
4,1000084126553197219,2025-01-13,2,80240,23600,NaN,NaN
...,...,...,...,...,...,...,...
4018069,999885621010689754,2026-01-01,4,119000,30000,85723,NaN
4018070,999885621010689754,2026-01-05,4,104000,30000,75558,NaN
4018071,999885621010689754,2026-01-06,4,104000,30000,75558,NaN
4018072,999885621010689754,2026-01-07,4,104000,30000,75558,NaN


In [ ]:
print(cal_pri.isnull().sum().to_string())

Country                         0
Location                        0
Airbnb_ListingID                0
Calendar_Date                   0
Calendar_Month                  0
Calendar_Year                   0
Stay_Week                       0
isAvailable                     0
Min_Nights                      0
Max_Nights                      0
Available_For_Checkin           0
Available_For_Checkout          0
RecordInserted                  0
Guests                          0
Price_Per_Night                 0
Cleaning_Fee                    0
Airbnb_Service_Fee         654855
Taxes                     3719787


In [ ]:
cal_pri.head()

,Country,Location,Airbnb_ListingID,Calendar_Date,Calendar_Month,Calendar_Year,Stay_Week,isAvailable,Min_Nights,Max_Nights,Available_For_Checkin,Available_For_Checkout,RecordInserted,Guests,Price_Per_Night,Cleaning_Fee,Airbnb_Service_Fee,Taxes
0,South Korea,Seoul,1000084126553197219,2025-01-06,1,2025,Weekday,True,3,365,True,True,2024-12-11 01:55:30.963228,2,80240,23600,NaN,NaN
1,South Korea,Seoul,1000084126553197219,2025-01-07,1,2025,Weekday,True,3,365,True,True,2024-12-11 01:55:30.963237,2,80240,23600,NaN,NaN
2,South Korea,Seoul,1000084126553197219,2025-01-08,1,2025,Weekday,True,3,365,True,True,2024-12-11 01:55:30.963245,2,80240,23600,NaN,NaN
3,South Korea,Seoul,1000084126553197219,2025-01-09,1,2025,Weekday,True,5,365,True,True,2024-12-11 01:55:30.963254,2,80240,23600,NaN,NaN
4,South Korea,Seoul,1000084126553197219,2025-01-13,1,2025,Weekday,True,3,365,True,True,2024-12-11 01:55:30.963289,2,80240,23600,NaN,NaN


In [ ]:
## review 전처리
review = review.dropna(subset=['Airbnb_ListingID'])
review['Airbnb_ListingID'] = pd.to_numeric(review['Airbnb_ListingID'], errors='coerce')
review['Rating'] = pd.to_numeric(review['Rating'], errors='coerce')
review = review.dropna(subset=['Rating'])
review['Rating'] = review['Rating'].astype(int)

<ipython-input-18-3f722727f5f4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review['Rating'] = review['Rating'].astype(int)


In [ ]:
columns_for_rag_summary = [
    # 위치 및 기본 정보
    'Airbnb_ListingID', 'Location', 'City', 'LocalizedNeighborhood','Lat','Lng'
    # 공간 구성
    'Bedrooms', 'Beds', 'Bathrooms', 'PersonCapacity',

    # 가격
    'Price_Per_Night', 'Cleaning_Fee', 'Taxes','total_price'

    # 숙소/공간 타입
    'RoomType_Clean', 'SpaceType_Clean',

    # 호스트 관련
    'Host_isSuperhost', 'Host_ResponseRate', 'Host_ResponseTime','HostLanguages_Verbose'

    # 편의시설
    'amenities_SelfCheckIn','CheckIn_Start','CheckOut_End'

    # 감성/분위기 관련 텍스트
    'ListingTitle_Localized', 'MainDescription_Localized', 'SpaceDescription_Localized',
    'PhotoCaptions_Localized', 'Host_Highlights', 'HostAbout_Localized',

    # 리뷰
    'ReviewScore', 'StarRating', 'ReviewCount', 'ReviewComments_Localized',

    # 기타
    'PictureCount'
]


# 1. 숙소 제목 정리

In [ ]:
des_over.head()

,Country_review,Location,Airbnb_ListingID,Lat,Lng,City,LocalizedCity,LocalizedNeighborhood,LocalizedNeighbourhood_ML,Title,...,Host_Highlights,PhotoCaptions,PhotoCaptions_Localized,Amenities,RecordInserted_desc,HostAbout,HostAbout_Localized,CheckIn_End,DuringStayDescription,DuringStayDescription_Localized
0,South Korea,Seoul,1231413898033280207,38,127,NaN,Seoul,Jung-gu,Jung-gu,The Very Central area of Seoul(time is money),...,"['My work: a laundry owner', 'Speaks English a...","['Full kitchen image 1', 'Full kitchen image 2...",[],"['Hair dryer', 'Cleaning products', 'Shampoo',...",2025-01-20 00:43:38.878762,"Good to hear from you,,, My name is Hyung seog...","Good to hear from you,,, My name is Hyung seog...",02:00,NaN,NaN
1,South Korea,Seoul,1256950542666270629,38,127,NaN,Seoul,Gwang-jin-gu,Gwang-jin-gu,프리돔,...,"['Lives in Seoul, South Korea']",[],[],"['Washing machine', 'TV', 'Air conditioning', ...",2025-01-20 00:44:01.958478,NaN,NaN,NaN,NaN,NaN
2,South Korea,Seoul,912141298876144195,38,127,NaN,Seoul,Jong-no-gu,Jong-no-gu,King's Garden (Open sales),...,"['Where I went to school: 대학교', 'Lives in Seou...","['single bedroom ', 'bedroom', 'single bedroom...",[],"['Hair dryer', 'Cleaning products', 'Hot water...",2025-01-20 00:44:03.440368,NaN,NaN,22:00,NaN,NaN
3,South Korea,Seoul,899479507487589813,38,127,NaN,Seoul,Ma-po-gu,Ma-po-gu,18.홍대입구역 10초 - city view,...,"['Born in the 80s', 'My work: 모험가']","['음식물 쓰레기 전용 봉투 :\n장기 숙박시, 전용 봉투에 담아 숙소 내 보관해주...",['Bags for food waste only:\nFor long-term sta...,"['City skyline view', 'Hair dryer', 'Cleaning ...",2025-01-20 00:43:58.438968,"안녕하세요. 더욱 즐거운 서울 여정이 되기위해 예쁜 숙소를 제공합니다.\n한국, 서...","Hello, I'll give you a pretty accommodation to...",02:00,NaN,NaN
4,South Korea,Seoul,1242579592234872945,38,127,NaN,Seoul,Gang-nam-gu,Gang-nam-gu,강남 선릉역 도보2분.NH. Full option.33,...,[],"['Living room image 1', 'Living room image 2',...",[],"['Hair dryer', 'Cleaning products', 'Body soap...",2025-01-20 00:44:18.478874,NaN,NaN,NaN,NaN,NaN


In [ ]:
title_df= des_over[['Airbnb_ListingID','Title']]
title_df['Title'] = title_df['Title'].fillna("")
title_df

<ipython-input-21-910b7d671254>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  title_df['Title'] = title_df['Title'].fillna("")


,Airbnb_ListingID,Title
0,1231413898033280207,The Very Central area of Seoul(time is money)
1,1256950542666270629,프리돔
2,912141298876144195,King's Garden (Open sales)
3,899479507487589813,18.홍대입구역 10초 - city view
4,1242579592234872945,강남 선릉역 도보2분.NH. Full option.33
...,...,...
19015,52855944,Ewha St Sinchon 3min Guest
19016,54010918,[DOUBLE] OpenHouse Apgujeong-rodeo Double Room
19017,854048278620397232,Dami's House
19018,658969258670527706,당산 한강뷰#Riverview #홍대 #Line No2&9 Dangsan Station


# 2. 숙소 위치 정리

In [ ]:
location_df = des_over[['Airbnb_ListingID', 'Location','Lat','Lng','LocalizedNeighbourhood_ML','LocationDescription']]
location_df['LocationDescription'] = location_df['LocationDescription'].fillna("")

<ipython-input-22-dc2dc93c505c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  location_df['LocationDescription'] = location_df['LocationDescription'].fillna("")


# 3. 숙소 정보 정리

In [ ]:
infor_df=des_over[['Airbnb_ListingID','MainDescription','SpaceDescription','GuestAccessDescription']]
infor_df['MainDescription'] = infor_df['MainDescription'].fillna("")
infor_df['SpaceDescription'] = infor_df['SpaceDescription'].fillna("")
infor_df['GuestAccessDescription'] = infor_df['GuestAccessDescription'].fillna("")
infor_df

<ipython-input-23-b73a3f4d6e2f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  infor_df['MainDescription'] = infor_df['MainDescription'].fillna("")
<ipython-input-23-b73a3f4d6e2f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  infor_df['SpaceDescription'] = infor_df['SpaceDescription'].fillna("")
<ipython-input-23-b73a3f4d6e2f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

,Airbnb_ListingID,MainDescription,SpaceDescription,GuestAccessDescription
0,1231413898033280207,* 4th floor and no elevator<br /><br />* you s...,* 4th floor without elevator<br /><br />* very...,"* 1 big double bed, 1 big single bed, 1 extra ..."
1,1256950542666270629,독특한 개성이 넘치는 이 로맨틱한 숙소가 마음에 드실 거예요.,,
2,912141298876144195,1. great location at the center of Seoul near ...,I have been dreaming to open the guesthouse an...,"Shared laundry room, shared kitchen (some room..."
3,899479507487589813,- 체크인 16시 / 체크아웃 정오 12시<br />- 밤10시 이후에는 소음정도를...,"큰 평수의 원룸 전체입니다.<br />고층 시티뷰로 낮은 물론, 서울 야경이 아름다...",♥ 숙소<br />- 침실 1: 침대 2개 (슈퍼싱글 + 슈퍼싱글)<br />- 주...
4,1242579592234872945,차분하면서도 스타일이 살아 있는 숙소에서 편안한 휴식을 즐기세요.<br />모든 것...,- 편의시설로 셋팅된 모던 하우스입니다.<br />- 퀸침대. S급 침구류로 셋팅되...,룸 공간 전체입니다.
...,...,...,...,...
19015,52855944,"It's your first time in Korea, right?<br />Wel...",Perect location<br /><br />- 3min : Ehwa Unive...,Private restroom in a room<br />TV<br />washer...
19016,54010918,"우리 숙소는 서울 강남의 핫 플레이스인 압구정로데오 거리 중심에 있어 세련된 맛집,...",,
19017,854048278620397232,We offer an additional low price for a long st...,"The warm accommodation has a bunk bed, providi...",The accommodation has a water purifier on each...
19018,658969258670527706,"욕실, 주방은 모두 공용이 아닌 개인 공간입니다. 공용 공간은 전혀 없습니다.",☆ 방<br />퀸 베드 1개<br />2인용 소파 1개<br />테이블&의자 2세...,** 대리석 테이블 사용시 바닥이 긁히지 않게 조심해 주세요. :)


# 4. 숙소 구성 정리

In [ ]:
from tqdm import tqdm

tqdm.pandas()
amenity_df = amenity.copy()

## 어메니티 컬럼들만 선택 (Airbnb_ListingID 및 기본 컬럼 제외)
amenity_cols = amenity_df.columns.difference(['Location', 'Country', 'Airbnb_ListingID', 'RecordInserted'])

## 어메니티가 있는 경우를 True로 바꾸기
amenity_binary = amenity_df[amenity_cols].fillna(0).astype(int)

## True인 컬럼 이름만 리스트로 추출
def get_amenities(row):
    return ', '.join([col for col in amenity_binary.columns if row[col] == 1])

amenity_df['Amenities'] = amenity_binary.progress_apply(get_amenities, axis=1)
amenity_df = amenity_df[['Airbnb_ListingID', 'Amenities']]

## amenity_df merge
amenity_df = amenity_df[['Airbnb_ListingID', 'Amenities']]
amenity_df = des_over[['Airbnb_ListingID', 'Bathrooms', 'Bedrooms', 'Beds', 'PersonCapacity']].merge(amenity_df, on = 'Airbnb_ListingID', how = 'inner')


100%|██████████| 40874/40874 [06:04<00:00, 112.21it/s]


In [ ]:
## 겹치는 숙소 ID 제거
amenity_df = amenity_df.drop_duplicates(subset='Airbnb_ListingID', keep='first')
amenity_df = amenity_df.fillna(0)
amenity_df

,Airbnb_ListingID,Bathrooms,Bedrooms,Beds,PersonCapacity,Amenities
0,1231413898033280207,1,1,3,4,"Bidet, Body soap, Carbon monoxide alarm, Centr..."
2,1256950542666270629,2,2,2,4,"Air conditioning, Carbon monoxide alarm, Dedic..."
4,912141298876144195,1,1,1,2,"Air conditioning, Bed linen, Carbon monoxide a..."
6,899479507487589813,1,3,2,4,"Air conditioning, Bed linen, Carbon monoxide a..."
8,1242579592234872945,1,1,1,1,"Air conditioning, Bed linen, Body soap, Carbon..."
...,...,...,...,...,...,...
37838,52855944,1,1,1,1,"Air conditioning, Bed linen, Body soap, Carbon..."
37840,54010918,1,1,1,2,"Air conditioning, Carbon monoxide alarm, Centr..."
37842,854048278620397232,1,1,2,3,"32-inch HDTV with Amazon Prime Video, Apple TV..."
37844,658969258670527706,1,1,1,2,"Air conditioning, Bed linen, Body soap, Carbon..."


# 5. 숙소 가격 정리


In [ ]:
# Calendar_Date를 기준으로 해야 예약여부랑 가격을 정리 할 수 있을듯
cal_pri.head()


,Country,Location,Airbnb_ListingID,Calendar_Date,Calendar_Month,Calendar_Year,Stay_Week,isAvailable,Min_Nights,Max_Nights,Available_For_Checkin,Available_For_Checkout,RecordInserted,Guests,Price_Per_Night,Cleaning_Fee,Airbnb_Service_Fee,Taxes
0,South Korea,Seoul,1000084126553197219,2025-01-06,1,2025,Weekday,True,3,365,True,True,2024-12-11 01:55:30.963228,2,80240,23600,NaN,NaN
1,South Korea,Seoul,1000084126553197219,2025-01-07,1,2025,Weekday,True,3,365,True,True,2024-12-11 01:55:30.963237,2,80240,23600,NaN,NaN
2,South Korea,Seoul,1000084126553197219,2025-01-08,1,2025,Weekday,True,3,365,True,True,2024-12-11 01:55:30.963245,2,80240,23600,NaN,NaN
3,South Korea,Seoul,1000084126553197219,2025-01-09,1,2025,Weekday,True,5,365,True,True,2024-12-11 01:55:30.963254,2,80240,23600,NaN,NaN
4,South Korea,Seoul,1000084126553197219,2025-01-13,1,2025,Weekday,True,3,365,True,True,2024-12-11 01:55:30.963289,2,80240,23600,NaN,NaN


In [ ]:
# 삭제할 컬럼 리스트
cols_to_drop = ['Country', 'Location', 'Calendar_Month', 'Calendar_Year', 'Stay_Week', 'RecordInserted', 'Taxes']

# 실제 존재하는 컬럼만 남기고 드롭
cal_pri.drop(columns=[col for col in cols_to_drop if col in cal_pri.columns], inplace=True)

In [ ]:
Airbnb_Servic_map = cal_pri.groupby('Airbnb_ListingID')['Airbnb_Service_Fee'].transform(
    lambda x: x.fillna(method='ffill').fillna(method='bfill')
)

# Step 2: 해당 값을 우선 채우고, 여전히 NaN이면 0으로 대체
cal_pri['Airbnb_Service_Fee'] = cal_pri['Airbnb_Service_Fee'].fillna(Airbnb_Servic_map)
cal_pri['Airbnb_Service_Fee'] = cal_pri['Airbnb_Service_Fee'].fillna(0)

<ipython-input-28-692ee42df52b>:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  lambda x: x.fillna(method='ffill').fillna(method='bfill')


In [ ]:
# 우선 Calendar_Date를 datetime 타입으로 변환
cal_pri['Calendar_Date'] = pd.to_datetime(cal_pri['Calendar_Date'])

def compress_by_month(sub_df):
    """Airbnb_ListingID별, 월별 예약 가능일 요약"""
    available_dates = sub_df[sub_df['isAvailable']]['Calendar_Date'].sort_values()

    if available_dates.empty:
        return ""

    available_dates = available_dates.dt.to_period('D')  # 날짜만 남기기
    month_groups = {}

    for date in available_dates:
        month = date.strftime('%m월')
        day = int(date.strftime('%d'))
        month_groups.setdefault(month, []).append(day)

    # 요약 포맷
    summary = []
    for month, days in month_groups.items():
        days.sort()
        compressed = []
        start = prev = days[0]
        for d in days[1:]:
            if d == prev + 1:
                prev = d
            else:
                compressed.append(f"{start}-{prev}" if start != prev else f"{start}")
                start = prev = d
        compressed.append(f"{start}-{prev}" if start != prev else f"{start}")
        summary.append(f"{month}: {', '.join(map(str, compressed))}")

    return "; ".join(summary)


# Airbnb_ListingID 별로 그룹화 후, Available_Ranges 컬럼 생성
available_ranges_df = cal_pri.groupby('Airbnb_ListingID').apply(compress_by_month).reset_index()
available_ranges_df.columns = ['Airbnb_ListingID', 'Available_Ranges']

# 원본 cal_pri에 merge
cal_pri = cal_pri.merge(available_ranges_df, on='Airbnb_ListingID', how='left')

<ipython-input-29-e0df5cfc977d>:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  available_ranges_df = cal_pri.groupby('Airbnb_ListingID').apply(compress_by_month).reset_index()


In [ ]:
cal_pri

,Airbnb_ListingID,Calendar_Date,isAvailable,Min_Nights,Max_Nights,Available_For_Checkin,Available_For_Checkout,Guests,Price_Per_Night,Cleaning_Fee,Airbnb_Service_Fee,Available_Ranges
0,1000084126553197219,2025-01-06,True,3,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27..."
1,1000084126553197219,2025-01-07,True,3,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27..."
2,1000084126553197219,2025-01-08,True,3,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27..."
3,1000084126553197219,2025-01-09,True,5,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27..."
4,1000084126553197219,2025-01-13,True,3,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27..."
...,...,...,...,...,...,...,...,...,...,...,...,...
3973448,999885621010689754,2025-12-29,True,3,365,True,True,4,119000,30000,85723,"01월: 20-23, 27, 27-28, 28-29, 29-30, 30; 02월: ..."
3973449,999885621010689754,2025-12-30,True,3,365,True,True,2,95000,30000,69460,"01월: 20-23, 27, 27-28, 28-29, 29-30, 30; 02월: ..."
3973450,999885621010689754,2025-12-30,True,3,365,True,True,4,119000,30000,85723,"01월: 20-23, 27, 27-28, 28-29, 29-30, 30; 02월: ..."
3973451,999885621010689754,2025-12-31,True,3,365,True,True,2,95000,30000,69460,"01월: 20-23, 27, 27-28, 28-29, 29-30, 30; 02월: ..."


In [ ]:
# 가격 컬럼들을 숫자형으로 변환
price = ['Price_Per_Night', 'Cleaning_Fee', 'Airbnb_Service_Fee']
for col in price:
    cal_pri[col] = pd.to_numeric(cal_pri[col].astype(str).str.replace(r'[^\d.]', '', regex=True), errors='coerce')

# 숙소별 평균 가격 계산
cal_pri_additional = cal_pri.groupby('Airbnb_ListingID').agg({
    'Price_Per_Night': 'mean',
    'Cleaning_Fee': 'mean',
    'Airbnb_Service_Fee': 'mean'
}).reset_index()

# 컬럼명 변경 (optional)
cal_pri_additional.rename(columns={
    'Price_Per_Night': 'Price_Per_Night_mean',
    'Cleaning_Fee': 'Cleaning_Fee_mean',
    'Airbnb_Service_Fee': 'Airbnb_Service_Fee_mean'
}, inplace=True)

# 원본 cal_pri에 merge
cal_pri = cal_pri.merge(cal_pri_additional, on='Airbnb_ListingID', how='left')


In [ ]:
cal_pri

,Airbnb_ListingID,Calendar_Date,isAvailable,Min_Nights,Max_Nights,Available_For_Checkin,Available_For_Checkout,Guests,Price_Per_Night,Cleaning_Fee,Airbnb_Service_Fee,Available_Ranges,Price_Per_Night_mean,Cleaning_Fee_mean,Airbnb_Service_Fee_mean
0,1000084126553197219,2025-01-06,True,3,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27...",99046,23600,0
1,1000084126553197219,2025-01-07,True,3,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27...",99046,23600,0
2,1000084126553197219,2025-01-08,True,3,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27...",99046,23600,0
3,1000084126553197219,2025-01-09,True,5,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27...",99046,23600,0
4,1000084126553197219,2025-01-13,True,3,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27...",99046,23600,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3973448,999885621010689754,2025-12-29,True,3,365,True,True,4,119000,30000,85723,"01월: 20-23, 27, 27-28, 28-29, 29-30, 30; 02월: ...",94108,30000,68855
3973449,999885621010689754,2025-12-30,True,3,365,True,True,2,95000,30000,69460,"01월: 20-23, 27, 27-28, 28-29, 29-30, 30; 02월: ...",94108,30000,68855
3973450,999885621010689754,2025-12-30,True,3,365,True,True,4,119000,30000,85723,"01월: 20-23, 27, 27-28, 28-29, 29-30, 30; 02월: ...",94108,30000,68855
3973451,999885621010689754,2025-12-31,True,3,365,True,True,2,95000,30000,69460,"01월: 20-23, 27, 27-28, 28-29, 29-30, 30; 02월: ...",94108,30000,68855


In [ ]:
cal_pri['total_price'] = cal_pri['Price_Per_Night'] + cal_pri['Cleaning_Fee'] + cal_pri['Airbnb_Service_Fee']

In [ ]:
cal_pri['total_price_mean'] = cal_pri['Price_Per_Night_mean'] + cal_pri['Cleaning_Fee_mean'] + cal_pri['Airbnb_Service_Fee_mean']

In [ ]:
cal_pri.head()

,Airbnb_ListingID,Calendar_Date,isAvailable,Min_Nights,Max_Nights,Available_For_Checkin,Available_For_Checkout,Guests,Price_Per_Night,Cleaning_Fee,Airbnb_Service_Fee,Available_Ranges,Price_Per_Night_mean,Cleaning_Fee_mean,Airbnb_Service_Fee_mean,total_price_mean,total_price
0,1000084126553197219,2025-01-06,True,3,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27...",99046,23600,0,122646,103840
1,1000084126553197219,2025-01-07,True,3,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27...",99046,23600,0,122646,103840
2,1000084126553197219,2025-01-08,True,3,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27...",99046,23600,0,122646,103840
3,1000084126553197219,2025-01-09,True,5,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27...",99046,23600,0,122646,103840
4,1000084126553197219,2025-01-13,True,3,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27...",99046,23600,0,122646,103840


# 6. 숙소 유형 정리

In [ ]:
## 겹치는 숙소 ID 제거
type_df = des_over[['Airbnb_ListingID', 'RoomType_Clean', 'SpaceType_Clean']]
type_df

,Airbnb_ListingID,RoomType_Clean,SpaceType_Clean
0,1231413898033280207,Other,Barn
1,1256950542666270629,Entire Home,Dome
2,912141298876144195,Other,Tower
3,899479507487589813,Other,Tower
4,1242579592234872945,Other,Tower
...,...,...,...
19015,52855944,Private Room,Room in serviced apartment
19016,54010918,Private Room,Room in serviced apartment
19017,854048278620397232,Private Room,Room in serviced apartment
19018,658969258670527706,Private Room,Room in serviced apartment


# 7. 슈퍼호스트 여부

In [ ]:
## 결측치 제거
superhost_df = des_over[['Airbnb_ListingID','isSuperhost', 'Host_ResponseRate', 'Host_ResponseTime','HostLanguages_Verbose']]
superhost_df['Host_ResponseRate'].fillna("", inplace=True)
superhost_df['Host_ResponseTime'].fillna("", inplace=True)
superhost_df['HostLanguages_Verbose'].fillna("", inplace=True)
superhost_df

<ipython-input-40-6a81ee2c4ccf>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  superhost_df['Host_ResponseRate'].fillna("", inplace=True)
<ipython-input-40-6a81ee2c4ccf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  superhost_df['Host_ResponseRate'].fillna("", inplace=True)
<ipython-input-40-6a81ee2c4ccf>:4: FutureWarning: A value is tryi

,Airbnb_ListingID,isSuperhost,Host_ResponseRate,Host_ResponseTime,HostLanguages_Verbose
0,1231413898033280207,False,100%,Responds within an hour,['English' 'Korean']
1,1256950542666270629,False,,Responds within a few days or more,[]
2,912141298876144195,False,90%,Responds within a few hours,[]
3,899479507487589813,True,100%,Responds within an hour,['Korean']
4,1242579592234872945,False,100%,Responds within an hour,[]
...,...,...,...,...,...
19015,52855944,False,100%,Responds within an hour,['English' 'Korean']
19016,54010918,False,100%,Responds within an hour,[]
19017,854048278620397232,True,86%,Responds within a few hours,['English' 'Korean']
19018,658969258670527706,True,100%,Responds within an hour,['Chinese' 'English' 'Korean']


# 8.편의성 정리

In [ ]:
checkin_df = des_over[['Airbnb_ListingID', 'amenities_SelfCheckIn', 'CheckIn_Start', 'CheckOut_End']]
# 결측치 채우기
checkin_df['amenities_SelfCheckIn'].fillna("", inplace=True)
checkin_df['CheckIn_Start'].fillna("", inplace=True)
checkin_df['CheckOut_End'].fillna("", inplace=True)

<ipython-input-41-f8020350c40d>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  checkin_df['amenities_SelfCheckIn'].fillna("", inplace=True)
<ipython-input-41-f8020350c40d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkin_df['amenities_SelfCheckIn'].fillna("", inplace=True)
<ipython-input-41-f8020350c40d>:4: FutureWarning: A value is 

# 7. 숙소 리뷰 정리

In [ ]:
import pandas as pd
import re

pd.set_option('display.max_colwidth', 100)

def clean_text(text):
    # 1. 한글, 숫자, 공백, 문장부호(.,?!만 남기기)
    text = re.sub(r'[^가-힣0-9\s\.\,\?\!]', ' ', str(text))

    # 2. 여러 공백 → 하나의 공백으로 줄이기
    text = re.sub(r'\s{2,}', ' ', text)

    # 3. 좌우 공백 제거
    return text.strip()

# ✅ 결측치 제거
review_df = review[['Airbnb_ListingID', 'ReviewLanguage', 'ReviewComments', 'Rating']]
review_df = review_df[review_df['ReviewLanguage'] == 'ko'].dropna(subset=['ReviewComments', 'Rating'])
review_df['ReviewComments'] = review_df['ReviewComments'].apply(clean_text)

# ✅ 리뷰 병합
grouped_comments = review_df.groupby('Airbnb_ListingID')['ReviewComments'].apply(lambda x: ' '.join(x)).reset_index()

# ✅ 평점 평균
grouped_rating = review_df.groupby('Airbnb_ListingID')['Rating'].mean().reset_index()

# ✅ 리뷰 개수
grouped_comments_counts = review_df.groupby('Airbnb_ListingID')['Rating'].size().reset_index()
grouped_comments_counts.columns = ['Airbnb_ListingID', 'ReviewCounts']

# ✅ 결합
review_summary_input = grouped_comments.merge(grouped_rating, on='Airbnb_ListingID') \
                                       .merge(grouped_comments_counts, on='Airbnb_ListingID')

# ✅ 리뷰 길이 필터
review_summary_input['Review_Length'] = review_summary_input['ReviewComments'].apply(lambda x: len(x))
review_summary_input = review_summary_input[review_summary_input['Review_Length'] > 0]

def generate_better_summary(text, max_len=100):
    if pd.isna(text) or not isinstance(text, str) or len(text.strip()) == 0:
        return ""

    # 문장 구분자로 자르기 (.!? 포함)
    sentences = re.split(r'(?<=[\.\!\?])\s+', text.strip())
    summary = ""

    # 문장이 1개 이상 있을 경우
    if len(sentences) >= 2:
        summary = sentences[0]
        # 문장이 너무 짧으면 두 번째 문장까지 포함
        if len(summary) < 30:
            summary += " " + sentences[1]
    elif len(sentences) == 1:
        summary = sentences[0]
    else:
        return text[:max_len] + '...'

    # 너무 길면 자르기
    return summary if len(summary) <= max_len else summary[:max_len].strip() + '...'
review_summary_input['ReviewSummary'] = review_summary_input['ReviewComments'].apply(generate_better_summary)


In [ ]:
review_summary_input

,Airbnb_ListingID,ReviewComments,Rating,ReviewCounts,Review_Length,ReviewSummary
0,69022,"위치, 뷰, 호스트님의 빠른 응대 뭐하나 빠짐없이 모든게 만족 스러운 스테이 였습니다. 감사합니다. 시골에서 할머니가 올라오셔서 서울 좋은 풍경을 보여드리고 싶어서 예약했는...",5,36,3461,"위치, 뷰, 호스트님의 빠른 응대 뭐하나 빠짐없이 모든게 만족 스러운 스테이 였습니다."
1,118591,너무 이쁘고 좋은방이었습니다 방음얘기많았는데 그런거 못느꼈어요 저희가시끄러웠으려나.. 친구들도 가격대비 너무이쁘고 좋다고 맘에들어했어요 특히 식탁이 넓은게 맘에들었습니다 다...,4,2,362,너무 이쁘고 좋은방이었습니다 방음얘기많았는데 그런거 못느꼈어요 저희가시끄러웠으려나..
2,265944,"잘쉬다 갑니다. 오랜만의 가족여행에 걸맞게 편안한 잠자리, 편리한 시설, 무엇보다도 탁월한 위치로 지하철도 편리하고 택시를 타도 편했다. 백화점과 쇼핑센타가 연결이 되어서 ...",5,6,844,"잘쉬다 갑니다. 오랜만의 가족여행에 걸맞게 편안한 잠자리, 편리한 시설, 무엇보다도 탁월한 위치로 지하철도 편리하고 택시를 타도 편했다."
3,305641,"전체적으로 깔금하고, 부대시설 또한 부족함 없고 신축에 가깝습니다. 약간 오르막길을 통한 위치에 있으나 마을버스도 있고, 걸어다녀도 될 정도고요. 만족할 만 합니다 잘자고갑...",4,4,335,"전체적으로 깔금하고, 부대시설 또한 부족함 없고 신축에 가깝습니다."
4,448840,4일동안 너무 감사했습니다 제 스케줄상 호스트님께 너무 불편하셨을텐데 많이 챙겨주시고 걱정도 해주시고 바쁜 하루를 보내는 여행에는 숙소 돌아가고 나서의 힐링이 되게 중요하는...,5,3,232,4일동안 너무 감사했습니다 제 스케줄상 호스트님께 너무 불편하셨을텐데 많이 챙겨주시고 걱정도 해주시고 바쁜 하루를 보내는 여행에는 숙소 돌아가고 나서의 힐링이 되게 중요하는...
...,...,...,...,...,...,...
14633,1328847523075032832,"성신여대 입구 근처에 위치한 숙소에서 정말 만족스러운 시간을 보냈습니다. 숙소는 매우 깨끗하고 정돈이 잘 되어 있었으며, 필요한 모든 시설이 잘 갖춰져 있었습니다. 주변 환...",5,1,276,성신여대 입구 근처에 위치한 숙소에서 정말 만족스러운 시간을 보냈습니다.
14634,1330463911160062464,잘 있다가 갑니다. 작지만 내부 깔끔해서 좋네요.,5,1,27,잘 있다가 갑니다. 작지만 내부 깔끔해서 좋네요.
14635,1330466631813913088,새로 리모델링해서 화장실이 정말 깨끗해요!! 방도 비좁지않고 침대도 편하고 둘이자기 딱좋아요! 합정 망원 걸어서 이동가능해서 서울에서 즐기고 잠자기 딱입니다,5,1,87,새로 리모델링해서 화장실이 정말 깨끗해요!! 방도 비좁지않고 침대도 편하고 둘이자기 딱좋아요!
14636,1330765159809291520,숙소도 생각보다 너무 깔끔하고 안에 내부도 넓어서 생각보다 좋았어요 ! 그리고 위치가 연남동 바로 앞에 위치해서 위치도 너무 좋았어요 잘 쉬다가 갑니다,5,1,84,숙소도 생각보다 너무 깔끔하고 안에 내부도 넓어서 생각보다 좋았어요 !


# 8. 종합

In [ ]:
final_df = title_df \
    .merge(location_df, on='Airbnb_ListingID', how='left') \
    .merge(infor_df, on='Airbnb_ListingID', how='left') \
    .merge(amenity_df, on='Airbnb_ListingID', how='left') \
    .merge(type_df, on='Airbnb_ListingID', how='left') \
    .merge(superhost_df, on='Airbnb_ListingID', how='left') \
    .merge(checkin_df, on='Airbnb_ListingID', how='left') \
    .merge(review_summary_input, on='Airbnb_ListingID', how='inner')


In [ ]:
final_df_all = cal_pri.merge(
    final_df,
    on='Airbnb_ListingID',
    how='left'
)


In [ ]:
## 리뷰 요약 Nan의 경우 그냥 리뷰로 사용
final_df_all['ReviewSummary'] = final_df_all['ReviewSummary'].fillna(final_df_all['ReviewComments'])

## 모든 결측치가 Hotel의 경우라서 Hotel로 대체
final_df_all['RoomType_Clean'] = final_df_all['RoomType_Clean'].fillna('Hotel')

## 이름, 리뷰가 없는 경우 제거
final_df_all = final_df_all.dropna(subset=['Title', 'ReviewComments'])

In [ ]:
## 리뷰 요약 Nan의 경우 그냥 리뷰로 사용
final_df['ReviewSummary'] = final_df['ReviewSummary'].fillna(final_df['ReviewComments'])

## 모든 결측치가 Hotel의 경우라서 Hotel로 대체
final_df['RoomType_Clean'] = final_df['RoomType_Clean'].fillna('Hotel')

## 이름, 리뷰가 없는 경우 제거
final_df = final_df.dropna(subset=['Title', 'ReviewComments'])

In [ ]:
# cal_pri의 컬럼 리스트 추출
columns_to_keep = cal_pri.columns.tolist()

# final_df_all에서 해당 컬럼만 추출
final_price_df = final_df_all[columns_to_keep]


In [ ]:
print("cal_pri shape:", cal_pri.shape)
print("final_df shape:", final_df.shape)
print("final_df_all shape:", final_df_all.shape)
print("final_price_df shape:", final_price_df.shape)

cal_pri shape: (3973453, 17)
final_df shape: (9759, 29)
final_df_all shape: (1922845, 45)
final_price_df shape: (1922845, 17)


In [ ]:
print(final_df.isnull().sum().to_string())

Airbnb_ListingID             0
Title                        0
Location                     0
Lat                          0
Lng                          0
LocalizedNeighbourhood_ML    0
LocationDescription          0
MainDescription              0
SpaceDescription             0
GuestAccessDescription       0
Bathrooms                    0
Bedrooms                     0
Beds                         0
PersonCapacity               0
Amenities                    0
RoomType_Clean               0
SpaceType_Clean              0
isSuperhost                  0
Host_ResponseRate            0
Host_ResponseTime            0
HostLanguages_Verbose        0
amenities_SelfCheckIn        0
CheckIn_Start                0
CheckOut_End                 0
ReviewComments               0
Rating                       0
ReviewCounts                 0
Review_Length                0
ReviewSummary                0


In [ ]:
final_df[final_df.Airbnb_ListingID == 48689803]

,Airbnb_ListingID,Title,Location,Lat,Lng,LocalizedNeighbourhood_ML,LocationDescription,MainDescription,SpaceDescription,GuestAccessDescription,...,Host_ResponseTime,HostLanguages_Verbose,amenities_SelfCheckIn,CheckIn_Start,CheckOut_End,ReviewComments,Rating,ReviewCounts,Review_Length,ReviewSummary
1015,48689803,블랙 노래방 바베큐 렌트룸 단체 모임 야외 테라스 캠핑장 홍대출구3분 3min st.,Seoul,38,127,Ma-po-gu,,"홍대 한복판 노래방과, 다트 바베큐까지!!<br /><br />✔ 숙박업소가 아니라 파티룸입니다.<br /><br />✔ 홍대입구역 도보 5분<br /><br />✔ 낮패키...",,,...,Responds within a few hours,[],False,19:00,10:00,주차가 안되는점 말고는 굿이었습니다. 다수의 인원이 가서 즐기기에 많은 컨텐츠들이 있어 좋았습니다 심야에 예약하고 왔는데 친구들과 너무너무 즐겁게 시간보내고 왔습니다. 소소...,5,8,390,주차가 안되는점 말고는 굿이었습니다. 다수의 인원이 가서 즐기기에 많은 컨텐츠들이 있어 좋았습니다 심야에 예약하고 왔는데 친구들과 너무너무 즐겁게 시간보내고 왔습니다.


In [ ]:
print(final_price_df .isnull().sum().to_string())

Airbnb_ListingID           0
Calendar_Date              0
isAvailable                0
Min_Nights                 0
Max_Nights                 0
Available_For_Checkin      0
Available_For_Checkout     0
Guests                     0
Price_Per_Night            0
Cleaning_Fee               0
Airbnb_Service_Fee         0
Available_Ranges           0
Price_Per_Night_mean       0
Cleaning_Fee_mean          0
Airbnb_Service_Fee_mean    0
total_price_mean           0
total_price                0


In [ ]:
final_price_df .head()

,Airbnb_ListingID,Calendar_Date,isAvailable,Min_Nights,Max_Nights,Available_For_Checkin,Available_For_Checkout,Guests,Price_Per_Night,Cleaning_Fee,Airbnb_Service_Fee,Available_Ranges,Price_Per_Night_mean,Cleaning_Fee_mean,Airbnb_Service_Fee_mean,total_price_mean,total_price
0,1000084126553197219,2025-01-06,True,3,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27; 03월: 3-6, 10-13, 17-20, 24-27, 31; 04월: 1-3, 7-1...",99046,23600,0,122646,103840
1,1000084126553197219,2025-01-07,True,3,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27; 03월: 3-6, 10-13, 17-20, 24-27, 31; 04월: 1-3, 7-1...",99046,23600,0,122646,103840
2,1000084126553197219,2025-01-08,True,3,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27; 03월: 3-6, 10-13, 17-20, 24-27, 31; 04월: 1-3, 7-1...",99046,23600,0,122646,103840
3,1000084126553197219,2025-01-09,True,5,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27; 03월: 3-6, 10-13, 17-20, 24-27, 31; 04월: 1-3, 7-1...",99046,23600,0,122646,103840
4,1000084126553197219,2025-01-13,True,3,365,True,True,2,80240,23600,0,"01월: 6-9, 13-16, 20-23; 02월: 3-6, 10-13, 24-27; 03월: 3-6, 10-13, 17-20, 24-27, 31; 04월: 1-3, 7-1...",99046,23600,0,122646,103840


In [ ]:
final_df.to_csv('/content/drive/MyDrive/dataset/final(0502).csv', index=False)

In [ ]:
print(final_df.isnull().sum().to_string())

Airbnb_ListingID             0
Title                        0
Location                     0
Lat                          0
Lng                          0
LocalizedNeighbourhood_ML    0
LocationDescription          0
MainDescription              0
SpaceDescription             0
GuestAccessDescription       0
Bathrooms                    0
Bedrooms                     0
Beds                         0
PersonCapacity               0
Amenities                    0
RoomType_Clean               0
SpaceType_Clean              0
isSuperhost                  0
Host_ResponseRate            0
Host_ResponseTime            0
HostLanguages_Verbose        0
amenities_SelfCheckIn        0
CheckIn_Start                0
CheckOut_End                 0
ReviewComments               0
Rating                       0
ReviewCounts                 0
Review_Length                0
ReviewSummary                0


In [ ]:
final_price_df.to_csv('/content/drive/MyDrive/dataset/final_price(0502).csv', index=False)

In [ ]:
print(final_price_df.isnull().sum().to_string())

Airbnb_ListingID           0
Calendar_Date              0
isAvailable                0
Min_Nights                 0
Max_Nights                 0
Available_For_Checkin      0
Available_For_Checkout     0
Guests                     0
Price_Per_Night            0
Cleaning_Fee               0
Airbnb_Service_Fee         0
Available_Ranges           0
Price_Per_Night_mean       0
Cleaning_Fee_mean          0
Airbnb_Service_Fee_mean    0
total_price_mean           0
total_price                0
